In [1]:
import numpy as np
import h5py
import sys


sys.path.append(".")
dirname = "./bgw_files/k-2x2x2/"

In [2]:
# epsinp and sigmain data.
from qtm.interfaces.bgw.epsinp import Epsinp
epsinp = Epsinp.from_epsilon_inp(filename=dirname+'epsilon.inp')

from qtm.interfaces.bgw.sigmainp import Sigmainp
sigmainp = Sigmainp.from_sigma_inp(filename=dirname+'sigma.inp')

In [3]:
from qtm.interfaces.bgw import inp
from qtm.interfaces.bgw.wfn2py import wfn2py

# Read wfndata
wfndata = wfn2py(dirname + "WFN.h5")
wfnqdata = wfn2py(dirname + "WFNq.h5")

# Read rho
rho = inp.read_rho(dirname + "RHO")

# Read vxc 
vxc = inp.read_vxc(dirname + "vxc.dat") 

In [4]:
from qtm.gw.core import QPoints
from qtm.gw.epsilon import Epsilon

epsilon = Epsilon.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)

Vcoul calculation for qpts: 100%|██████████| 8/8 [00:00<00:00, 14755.69it/s]


In [5]:
from tqdm import trange
from qtm.gw.core import reorder_2d_matrix_sorted_gvecs, sort_cryst_like_BGW


def calculate_epsilon(numq=None, writing=False):
    epsmats = []
    if numq is None:
        numq = epsilon.qpts.numq

    for i_q in trange(0, numq, desc="Epsilon> q-pt index"):
        # Create map between BGW's sorting order and QTm's sorting order
        gkspc = epsilon.l_gq[i_q]
        
        if i_q == epsilon.qpts.index_q0:
            key = gkspc.g_norm2
        else:
            key = gkspc.gk_norm2

        indices_gspace_sorted = sort_cryst_like_BGW(
            cryst=gkspc.g_cryst, key_array=key
        )
       

        # Calculate matrix elements
        M = next(epsilon.matrix_elements(i_q=i_q))

        # Calculate polarizability matrix (faster, but not memory-efficient)
        chimat = epsilon.polarizability(M)

        # Calculate polarizability matrix (memory-efficient)
        # chimat = epsilon.polarizability_active(i_q)

        # Calculate epsilon inverse matrix
        epsinv0 = epsilon.epsilon_inverse(i_q=i_q, polarizability_matrix=chimat, store=True)
        epsinv = reorder_2d_matrix_sorted_gvecs(epsinv0, indices_gspace_sorted)

        epsilon.l_epsinv[i_q] = epsinv

        
        # Compare the results with BGW's results
        if i_q == epsilon.qpts.index_q0:
            epsref = epsilon.read_epsmat(dirname + "eps0mat.h5")[0][0, 0]
            if writing:
                epsilon.write_epsmat(
                    filename="test/epsilon/eps0mat_qtm.h5", epsinvmats=[epsinv]
                )
        else:
            epsref = np.array(epsilon.read_epsmat(dirname + "epsmat.h5")[i_q - 1][0, 0])
            epsmats.append(epsinv)

        # Calculate stddev between reference and calculated epsinv matrices
        std_eps = np.std(epsref - epsinv) / np.sqrt(np.prod(list(epsinv.shape)))


        epstol = 1e-16
        if np.abs(std_eps) > epstol:
            print(f"Standard deviation exceeded {epstol} tolerance: {std_eps}, for i_q:{i_q}")


    if writing:
        epsilon.write_epsmat(filename="test/epsilon/epsmat_qtm.h5", epsinvmats=epsmats)


epsinp.no_min_fftgrid = True
epsilon = Epsilon.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)

calculate_epsilon()

Epsilon> q-pt index: 100%|██████████| 8/8 [00:00<00:00, 122.75it/s]

Standard deviation exceeded 1e-16 tolerance: 2.5494915419483335e-15, for i_q:0


In [6]:
from kernel import KernelMtxEl

q0val = [0.001, 0.001, 0.001]


In [7]:
kernelclass = KernelMtxEl.from_BGW(
    wfndata=wfndata,
    epsinp=epsinp,
    sigmainp=sigmainp,
    l_epsmats=epsilon.l_epsinv,
    q0val=q0val,
    parallel=False,
    num_bands_val=1,
    num_bands_con=1,
)

In [8]:
data = kernelclass.calc_kernel_mtxel()

In [9]:
exc = data["exc"]
print(exc.shape)
exc = exc[:, :, 0, 0, 0, 0]

for row in range(exc.shape[0]):
    row_data = []
    for col in range(exc.shape[1]):
        row_data.append(f"{exc[row, col]:.6f}")
    print(" | ".join(row_data))

(8, 8, 1, 1, 1, 1)
0.478812-0.000000j | -0.029943-0.114178j | -0.070840-0.023855j | -0.069334+0.124695j | -0.102824+0.157271j | 0.016159-0.055476j | 0.023062+0.090879j | 0.000498-0.000065j
-0.029943-0.114178j | 0.216560+0.000000j | 0.023217-0.035262j | -0.013489-0.012930j | 0.027856+0.030818j | -0.015734-0.009451j | -0.100181-0.000795j | -0.002107+0.015270j
-0.070840-0.023855j | 0.023217-0.035262j | 0.216558+0.000000j | 0.003350-0.018209j | -0.010606+0.040923j | 0.004145+0.100101j | 0.010076+0.015572j | 0.000492-0.000236j
-0.069334+0.124695j | -0.013489-0.012930j | 0.003350-0.018209j | 0.242285-0.000000j | 0.099929+0.007170j | 0.006418-0.001458j | -0.004717+0.004452j | 0.050097+0.069127j
-0.102824+0.157271j | 0.027856+0.030818j | -0.010606+0.040923j | 0.099929+0.007170j | 0.216560+0.000000j | -0.017545+0.005331j | 0.012645-0.013767j | -0.009655-0.011491j
0.016159-0.055476j | -0.015734-0.009451j | 0.004145+0.100101j | 0.006418-0.001458j | -0.017545+0.005331j | 0.242346+0.000000j | -0.00

In [10]:
head = data["head"]
head = head[:, :, 0, 0, 0, 0]

for row in range(head.shape[0]):
    row_data = []
    for col in range(head.shape[1]):
        row_data.append(f"{head[row, col]:.6f}")
    print(" | ".join(row_data))

1.000000-0.000000j | -0.050160-0.191313j | -0.113255-0.038140j | -0.123287+0.217328j | -0.179906+0.275184j | 0.028428-0.098212j | 0.043078+0.159901j | -0.000001+0.000000j
-0.050160-0.191313j | 1.000000+0.000000j | 0.044808-0.068063j | -0.240968-0.233186j | 0.053185+0.058837j | -0.281817-0.170101j | -0.031007+0.000899j | -0.000242+0.001754j
-0.113255-0.038140j | 0.044808-0.068063j | 1.000000-0.000000j | 0.060722-0.326442j | -0.020488+0.079056j | 0.001420+0.030969j | 0.181951+0.278390j | 0.000166-0.000079j
-0.123287+0.217328j | -0.240968-0.233186j | 0.060722-0.326442j | 1.000000-0.000000j | 0.030873+0.002878j | -0.043087+0.008548j | 0.032040-0.029986j | 0.016715+0.020475j
-0.179906+0.275184j | 0.053185+0.058837j | -0.020488+0.079056j | 0.030873+0.002878j | 1.000000+0.000000j | -0.314413+0.096719j | 0.225249-0.248664j | -0.001213-0.001443j
0.028428-0.098212j | -0.281817-0.170101j | 0.001420+0.030969j | -0.043087+0.008548j | -0.314413+0.096719j | 1.000000+0.000000j | 0.037470-0.022926j | 0

In [11]:
wings = data["wings"]
wings = wings[:, :, 0, 0, 0, 0]

for row in range(wings.shape[0]):
    row_data = []
    for col in range(wings.shape[1]):
        row_data.append(f"{wings[row, col]:.6f}")
    print(" | ".join(row_data))

0.000000+0.000000j | -0.001630-0.006218j | -0.033641-0.011334j | 0.000838-0.001620j | -0.002887+0.004431j | -0.008970+0.030424j | -0.001245-0.006001j | -0.000501+0.000061j
-0.001630-0.006218j | 0.000000+0.000000j | 0.001529-0.002321j | -0.073974-0.071189j | -0.004187-0.004632j | -0.234632-0.141222j | -0.000594+0.000061j | 0.000127-0.000924j
-0.033641-0.011334j | 0.001529-0.002321j | 0.000000+0.000000j | -0.010940+0.056295j | -0.003399+0.013120j | 0.000362+0.003931j | 0.099601+0.154095j | 0.000597-0.000285j
0.000838-0.001620j | -0.073974-0.071189j | -0.010940+0.056295j | 0.000000+0.000000j | 0.000151-0.000011j | -0.006847+0.001714j | 0.001567-0.001357j | -0.010412-0.014241j
-0.002887+0.004431j | -0.004187-0.004632j | -0.003399+0.013120j | 0.000151-0.000011j | 0.000000+0.000000j | 0.090088-0.027280j | 0.071578-0.078588j | -0.000163-0.000194j
-0.008970+0.030424j | -0.234632-0.141222j | 0.000362+0.003931j | -0.006847+0.001714j | 0.090088-0.027280j | 0.000000+0.000000j | -0.000161+0.000197j

In [12]:
body = data["body"]
body = body[:, :, 0, 0, 0, 0]

for row in range(body.shape[0]):
    row_data = []
    for col in range(body.shape[1]):
        row_data.append(f"{body[row, col]:.6f}")
    print(" | ".join(row_data))

0.317954+0.000001j | -0.069684-0.265868j | -0.120007-0.040354j | -0.193866+0.351082j | -0.312357+0.477733j | 0.032626-0.113615j | 0.053418+0.213881j | 0.006985-0.000845j
-0.069684-0.265868j | 0.124536-0.000001j | 0.170755-0.259334j | -0.235681-0.224035j | 0.138472+0.153207j | -0.448113-0.267274j | -0.064149-0.002121j | -0.012140+0.087979j
-0.120007-0.040354j | 0.170755-0.259334j | 0.115235-0.000001j | 0.062344-0.345201j | -0.071068+0.274260j | 0.002398+0.065936j | 0.276534+0.430320j | 0.009154-0.004369j
-0.193866+0.351082j | -0.235681-0.224035j | 0.062344-0.345201j | 0.015061-0.000059j | 0.053467+0.003194j | 0.087028-0.017733j | -0.015762+0.014909j | 0.028378+0.042751j
-0.312357+0.477733j | 0.138472+0.153207j | -0.071068+0.274260j | 0.053467+0.003194j | 0.155066+0.000002j | -0.591210+0.177398j | 0.350255-0.378564j | -0.033546-0.039926j
0.032626-0.113615j | -0.448113-0.267274j | 0.002398+0.065936j | 0.087028-0.017733j | -0.591210+0.177398j | 0.026862+0.000862j | -0.112390+0.066353j | -0

In [13]:
l_epsinv = epsilon.l_epsinv
eps0mat = l_epsinv[0]

# for row in range(0, eps0mat.shape[0]):
#     for col in range(0, eps0mat.shape[1]):
#         print(f"eps0mat[{row}, {col}] = {eps0mat[row, col]}")


In [14]:
l_epsinv_sorted = kernelclass.l_epsinv
eps0mat_sorted = l_epsinv_sorted[0]

# for row in range(0, eps0mat_sorted.shape[0]):
#     for col in range(0, eps0mat_sorted.shape[1]):
#         print(f"eps0mat_sorted[{row}, {col}] = {eps0mat_sorted[row, col]}")